## Segmenting and Clustering Neighborhoods in Los Angeles


In [1]:
#import numpy
#print(numpy)

#!pip3 uninstall numpy --yes
#!pip3 install numpy==1.18.4

In [2]:
#!pip uninstall numpy
#!pip install -U numpy

In [ ]:
!conda update pandas
!pip install beautifulsoup4
!pip install lxml


#!conda install -U numpy --yes
!pip install -U numpy
!pip install -U scipy
!pip install -U scikit-learn
!pip install -U imbalanced-learn
!pip install folium


# library to handle data in a vectorized manner
import numpy as np 

# library for data analsysis
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# library to handle requests
import requests 
# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

from IPython.display import Image 
from IPython.core.display import HTML 

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
# map rendering library
import folium 


print('Libraries imported.')

Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | 

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

# Convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 
# Tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize  

# Import folium as the map rendering library
import folium 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Scrape Site

In [ ]:
# Code to Scrape the Canadian Postal code table from its Wikipedia page
source = requests.get('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Houston').text
soup=BeautifulSoup(source,'lxml')
print(soup.title)
from IPython.display import display_html
tab = str(soup.table)
display_html(tab,raw=True)

In [ ]:
dfs = pd.read_html(tab)
df=dfs[0]
df.head()

In [ ]:
# The rows will be dropped when Borough is marked as 'Not assigned' 
df1 = df[df.Borough != 'Not assigned']

# Neighbourhoods will be combined is they have the same Postalcode
df2 = df1.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)

# Neighbourhoods will have 'n' to mark their Borough name is if it was 'Not assigned'
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned',df2['Borough'], df2['Neighbourhood'])

df2

In [ ]:
df2.shape

In [ ]:
df2.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df2.head()

In [ ]:
df2.rename(columns={'Postalcode':'Postcode'}, inplace=True)
df2.head()

In [ ]:
# Code to import the csv file that holds the latitudes and longitudes for various neighbourhoods
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

In [ ]:
lat_lon.rename(columns={'Postal Code':'Postcode'},inplace=True)
df3 = pd.merge(df2,lat_lon,on='Postcode')
df3.head()

In [ ]:
df4 = df3[df3['Borough'].str.contains('Houston',regex=False)]
df4

In [ ]:
map_Houston = folium.Map(location=[43.651070,-79.347015],zoom_start=12)

for lat,lng,borough,neighbourhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_Houston)
map_Houston


In [ ]:
k=5
Houston_clustering = df4.drop(['Postcode','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(Houston_clustering)
kmeans.labels_
df4.insert(0, 'Cluster Label', kmeans.labels_)

In [ ]:
df4

In [ ]:
# Creating a map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=13)

# Set the cluster color schemes
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add map markers
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df4['Latitude'], df4['Longitude'], df4['Neighbourhood'], df4['Cluster Label']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
#!pip install -U numpy

#!pip install -U pandas

#!pip install -U scipy

#!pip install -U scikit-learn

#!pip install -U imbalanced-learn